In [5]:
from db_functions import db_create_engine
import pandas as pd
from configparser import ConfigParser
import sys

print('Pandas', pd.__version__)

Pandas 0.20.3


In [7]:
engine = db_create_engine(config_file='config.ini',
                          conn_name='AWS_RDS')

In [8]:
statement = 

SyntaxError: invalid syntax (<ipython-input-8-998e5afd54d4>, line 1)

In [ ]:
data = pd.read_sql_query('''SELECT * FROM allstarfull LIMIT 5''', cnx)